# **Popular Chess Openings based on Rating and Time Control**




This Python Notebook aims to gather the most popular chess opening repertoire depending on the player's rating and chosen time control, which will help the player know what opening is played the most at their specific level and time format. By analyzing a large dataset of chess games from various online platforms, this project will identify patterns and trends in opening choices among players of different skill levels (e.g., beginner, intermediate, advanced) and across different time controls (e.g., classical, rapid, blitz). The insights derived from this analysis can provide players with a strategic advantage by informing them of commonly encountered openings in their games, enabling them to add these to their arsenal and improve their overall performance.

## **Importing libraries**

In [1]:
import pandas as pd
import numpy as np
import chess.pgn



## **Parsing the PGN file**

The games found in the PGN file is extracted from the database of FICS (Free Internet Chess Server). The link to the website can be found [here](https://www.ficsgames.org/download.html). The games extracted were from the entire year of 2023, and it included every time control and every player rating.  

In [2]:
# # Initialize an empty dictionary to store the game data
# game_data = {
#     "Result": [],
#     "WhiteElo": [], 
#     "BlackElo": [],
#     "TimeControl": [], 
#     "ECO": []
# }

# num_games = 1

# with open("./dataset/2023_fics_games.pgn") as pgn_file:
#     # Parse the first game in the file
#     game = chess.pgn.read_game(pgn_file)
#     print(game.headers)

#     # Iterate through all games in the file
#     while game is not None:
#         print(f"Processing game {num_games}", end="\r")
#         # Append game headers (metadata) to the game_data dictionary
#         for key in game_data.keys():
#             if key in game.headers:
#                 game_data[key].append(game.headers[key])
#             else:
#                 game_data[key].append(None)    
#         # Read the next game in the file
#         game = chess.pgn.read_game(pgn_file)
#         num_games += 1
#         if num_games > 1000000:
#             break
# print(f"Done parsing the games. Total games: {num_games}")

# # Convert the game_data dictionary into a DataFrame
# df = pd.DataFrame(game_data)
# df.head()

Before we proceed, let's save this dataframe into a `.csv` file. 

In [3]:
# df.to_csv("./dataset/2023_fics_games.csv", index=False)

## **Preprocessing the dataset**

The dataset used for this project comprises approximately 1,000,000 chess games that were played on ficgames.org. Each game record includes details such as the result, time control, player ratings, and opening ECO code.

In [4]:
df = pd.read_csv("./dataset/2023_fics_games.csv") 
df.head()

,Result,WhiteElo,BlackElo,TimeControl,ECO
0,1-0,1491,1554,blitz,D58
1,0-1,1520,1458,rapid,B02
2,1-0,1613,1553,rapid,C00
3,1-0,1338,1411,blitz,A40
4,0-1,1504,1608,blitz,D02


Let's see the shape of the dataset. 

In [5]:
df.shape

(999990, 5)

Let's also see the number of columns and its datatype. 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999990 entries, 0 to 999989
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Result       999990 non-null  object
 1   WhiteElo     999990 non-null  int64 
 2   BlackElo     999990 non-null  int64 
 3   TimeControl  999990 non-null  object
 4   ECO          999990 non-null  object
dtypes: int64(2), object(3)
memory usage: 38.1+ MB


We can see that there are rows that have null values on the `ECO` column. Let's simply drop these. 

In [7]:
df.dropna(subset=['ECO'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999990 entries, 0 to 999989
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Result       999990 non-null  object
 1   WhiteElo     999990 non-null  int64 
 2   BlackElo     999990 non-null  int64 
 3   TimeControl  999990 non-null  object
 4   ECO          999990 non-null  object
dtypes: int64(2), object(3)
memory usage: 38.1+ MB


Note that I haved grouped the original values in `TimeControl` (i.e. 180+0, 120+12) into `classical`, `blitz`, `rapid`, and `bullet` for better readability. The grouping was based on what is used by lichess.org, which can be seen [here](https://lichess.org/faq#time-controls). 

The estimated game duration will be computed using this formula: 

`estimated game duration = (clock initial time) + 40 × (clock increment)`

This was how the resulting duration will be grouped:
* < 179s = `bullet`
* < 479s = `blitz`
* < 1500s = `rapid`
* ≥ 1500s = `classical`

Before we proceed, let's see the number of games played using a specific time control. 

In [8]:
df['TimeControl'].value_counts()

TimeControl
blitz        759410
rapid        209144
bullet        24106
classical      7330
Name: count, dtype: int64

Now, let's group the ratings based on this criteria found in this [forum](https://lichess.org/forum/general-chess-discussion/what-is-a-good-lichess-rating) on Lichess:
* < 1300 = `beginner`
* < 2000 = `intermediate`
* < 2400 = `advanced`
* < 2700 = `master`
* ≥ 2700 = `grandmaster`

In [9]:
white_rating_conditions = [
    (df['WhiteElo'] < 1300), 
    (df['WhiteElo'] >= 1300) & (df['WhiteElo'] < 2000),
    (df['WhiteElo'] >= 2000) & (df['WhiteElo'] < 2700),
    (df['WhiteElo'] >= 2700)
]

black_rating_conditions = [
    (df['BlackElo'] < 1300), 
    (df['BlackElo'] >= 1300) & (df['BlackElo'] < 2000),
    (df['BlackElo'] >= 2000) & (df['BlackElo'] < 2700),
    (df['BlackElo'] >= 2700)
]

rating_choices = ['beginner', 'intermediate', 'advanced', 'grandmaster']

df['WhiteLevel'] = np.select(white_rating_conditions, rating_choices, default='unknown')
df['BlackLevel'] = np.select(black_rating_conditions, rating_choices, default='unknown')
df.head()

,Result,WhiteElo,BlackElo,TimeControl,ECO,WhiteLevel,BlackLevel
0,1-0,1491,1554,blitz,D58,intermediate,intermediate
1,0-1,1520,1458,rapid,B02,intermediate,intermediate
2,1-0,1613,1553,rapid,C00,intermediate,intermediate
3,1-0,1338,1411,blitz,A40,intermediate,intermediate
4,0-1,1504,1608,blitz,D02,intermediate,intermediate


Now that there is a `WhiteLevel` and `BlackLevel` columns, we can now remove the `WhiteElo` and `BlackElo` columns. 

In [10]:
df = df.drop(["WhiteElo", "BlackElo"], axis=1)
df.head()

,Result,TimeControl,ECO,WhiteLevel,BlackLevel
0,1-0,blitz,D58,intermediate,intermediate
1,0-1,rapid,B02,intermediate,intermediate
2,1-0,rapid,C00,intermediate,intermediate
3,1-0,blitz,A40,intermediate,intermediate
4,0-1,blitz,D02,intermediate,intermediate


Let's remove the rows in which black and white belong to different groups in terms of rating. 

In [11]:
df_same_rating = df.loc[df['WhiteLevel'] == df['BlackLevel']]
df_same_rating.shape

(841570, 5)

Let's see the number of games played depending on the level of the players. 

In [12]:
df_games_played_per_level = df_same_rating['WhiteLevel'].value_counts().reset_index()
df_games_played_per_level.columns = ['Level', 'Count']
df_games_played_per_level

,Level,Count
0,intermediate,800189
1,beginner,28772
2,advanced,11368
3,grandmaster,1241


`grandmaster` hass too few games played. Let's combine them with the `advanced`. 

In [13]:
df_same_rating['WhiteLevel'] = df_same_rating['WhiteLevel'].where(df_same_rating['WhiteLevel'] != "grandmaster", "advanced")
df_same_rating['BlackLevel'] = df_same_rating['BlackLevel'].where(df_same_rating['BlackLevel'] != "grandmaster", "advanced")
df_games_played_per_level = df_same_rating['WhiteLevel'].value_counts().reset_index()
df_games_played_per_level.columns = ['Level', 'Count']
df_games_played_per_level

/tmp/ipykernel_747568/651671217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_same_rating['WhiteLevel'] = df_same_rating['WhiteLevel'].where(df_same_rating['WhiteLevel'] != "grandmaster", "advanced")
/tmp/ipykernel_747568/651671217.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_same_rating['BlackLevel'] = df_same_rating['BlackLevel'].where(df_same_rating['BlackLevel'] != "grandmaster", "advanced")


,Level,Count
0,intermediate,800189
1,beginner,28772
2,advanced,12609


Let's also see the number of games played based on the opening. 

In [14]:
df_games_played_per_opening = df_same_rating['ECO'].value_counts().reset_index()
df_games_played_per_opening.columns = ['ECO', 'Count']
df_games_played_per_opening.sort_values(by='ECO', inplace=True)
df_games_played_per_opening

,ECO,Count
0,A00,57622
23,A01,8272
35,A02,6334
44,A03,4681
14,A04,11205
...,...,...
387,E95,24
487,E96,1
234,E97,159
374,E98,30


The `Result` column is encoded as `1-0` when white wins, `0-1` when black wins, and `1/2-1/2` when the game is a draw. Let's change that to indicate who won or if it's a draw to make it more readable. 

In [15]:
win_conditions = [
    (df_same_rating['Result'] == '1-0'),
    (df_same_rating['Result'] == '0-1'),
    (df_same_rating['Result'] == '1/2-1/2')
]

win_choices = ['white', 'black', 'draw']

df_same_rating['Result'] = np.select(win_conditions, win_choices, default='unknown')
df_same_rating.head()

/tmp/ipykernel_747568/117476992.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_same_rating['Result'] = np.select(win_conditions, win_choices, default='unknown')


,Result,TimeControl,ECO,WhiteLevel,BlackLevel
0,white,blitz,D58,intermediate,intermediate
1,black,rapid,B02,intermediate,intermediate
2,white,rapid,C00,intermediate,intermediate
3,white,blitz,A40,intermediate,intermediate
4,black,blitz,D02,intermediate,intermediate


Finally, let's reorder the columns and rows for better readability. 

In [16]:
df_preprocessed = df_same_rating.reindex(columns=['Result', 'WhiteLevel', 'BlackLevel', 'TimeControl', 'ECO'])
df_preprocessed.reset_index(drop=True, inplace=True)
df_preprocessed.head()

,Result,WhiteLevel,BlackLevel,TimeControl,ECO
0,white,intermediate,intermediate,blitz,D58
1,black,intermediate,intermediate,rapid,B02
2,white,intermediate,intermediate,rapid,C00
3,white,intermediate,intermediate,blitz,A40
4,black,intermediate,intermediate,blitz,D02


The data preprocessing is now done! Let's save it into a new `.csv` file. 

In [17]:
df_preprocessed.to_csv("./dataset/cleaned_dataset.csv", index=False)

## **Data Exploration**

Now that the dataset is cleaned, we can now explore the data and see if we can extract useful information from it. First, let's gather the numbers. 

In [18]:
df_games_played_per_opening = df_preprocessed['ECO'].value_counts().reset_index()
df_games_played_per_opening.columns = ['ECO', 'Count']
df_games_played_per_opening.sort_values(by='ECO', inplace=True)
df_games_played_per_opening

,ECO,Count
0,A00,57622
23,A01,8272
35,A02,6334
44,A03,4681
14,A04,11205
...,...,...
387,E95,24
487,E96,1
234,E97,159
374,E98,30


In [19]:
df_games_played_per_level = df_preprocessed['WhiteLevel'].value_counts().reset_index()
df_games_played_per_level.columns = ['Level', 'Count']
df_games_played_per_level

,Level,Count
0,intermediate,800189
1,beginner,28772
2,advanced,12609


In [20]:
df_games_played_per_time_control = df_preprocessed['TimeControl'].value_counts().reset_index()
df_games_played_per_time_control.columns = ['TimeControl', 'Count']
df_games_played_per_time_control

,TimeControl,Count
0,blitz,674929
1,rapid,144421
2,bullet,16295
3,classical,5925


### **Time Control and Player Level**

Let's now see how many `blitz`, `rapid`, `bullet`, and `classical` games are played by `beginner`, `intermediate`, and `advanced` players. 

#### **Time Control: Blitz**

In [21]:
df_blitz_beginner = df_preprocessed.loc[(df_preprocessed['TimeControl'] == 'blitz') & (df_preprocessed['WhiteLevel'] == 'beginner')]
df_blitz_beginner.shape

(10998, 5)

In [22]:
df_blitz_intermediate = df_preprocessed.loc[(df_preprocessed['TimeControl'] == 'blitz') & (df_preprocessed['WhiteLevel'] == 'intermediate')]
df_blitz_intermediate.shape

(656859, 5)

In [23]:
df_blitz_advanced = df_preprocessed.loc[(df_preprocessed['TimeControl'] == 'blitz') & (df_preprocessed['WhiteLevel'] == 'advanced')]
df_blitz_advanced.shape

(7072, 5)

#### **Time Control: Rapid**

In [24]:
df_rapid_beginner = df_preprocessed.loc[(df_preprocessed['TimeControl'] == 'rapid') & (df_preprocessed['WhiteLevel'] == 'beginner')]
df_rapid_beginner.shape

(17694, 5)

In [25]:
df_rapid_intermediate = df_preprocessed.loc[(df_preprocessed['TimeControl'] == 'rapid') & (df_preprocessed['WhiteLevel'] == 'intermediate')]
df_rapid_intermediate.shape

(123458, 5)

In [26]:
df_rapid_advanced = df_preprocessed.loc[(df_preprocessed['TimeControl'] == 'rapid') & (df_preprocessed['WhiteLevel'] == 'advanced')]
df_rapid_advanced.shape

(3269, 5)

#### **Time Control: Bullet**

In [27]:
df_bullet_beginner = df_preprocessed.loc[(df_preprocessed['TimeControl'] == 'bullet') & (df_preprocessed['WhiteLevel'] == 'beginner')]
df_bullet_beginner.shape

(42, 5)

In [28]:
df_bullet_intermediate = df_preprocessed.loc[(df_preprocessed['TimeControl'] == 'bullet') & (df_preprocessed['WhiteLevel'] == 'intermediate')]
df_bullet_intermediate.shape

(15013, 5)

In [29]:
df_bullet_advanced = df_preprocessed.loc[(df_preprocessed['TimeControl'] == 'bullet') & (df_preprocessed['WhiteLevel'] == 'advanced')]
df_bullet_advanced.shape

(1240, 5)

#### **Time Control: Classical**

In [30]:
df_classical_beginner = df_preprocessed.loc[(df_preprocessed['TimeControl'] == 'classical') & (df_preprocessed['WhiteLevel'] == 'beginner')]
df_classical_beginner.shape

(38, 5)

In [31]:
df_classical_intermediate = df_preprocessed.loc[(df_preprocessed['TimeControl'] == 'classical') & (df_preprocessed['WhiteLevel'] == 'intermediate')]
df_classical_intermediate.shape

(4859, 5)

In [32]:
df_classical_advanced = df_preprocessed.loc[(df_preprocessed['TimeControl'] == 'classical') & (df_preprocessed['WhiteLevel'] == 'advanced')]
df_classical_advanced.shape

(1028, 5)

### **Openings Played and Win Percentage**

Now, let's see what openings are played and their respective win percentage for each time control and player level. 

#### **Time Control: Blitz**

In [33]:
df_opening_blitz_beginner = df_blitz_beginner['ECO'].value_counts().reset_index()
df_opening_blitz_beginner.columns = ['ECO', 'TotalGames']

white_wins = df_blitz_beginner.loc[df_blitz_beginner['Result'] == 'white', 'ECO'].value_counts().reset_index()
white_wins.columns = ['ECO', 'WhiteWins']
df_opening_blitz_beginner = df_opening_blitz_beginner.merge(white_wins, on='ECO', how='left')

black_wins = df_blitz_beginner.loc[df_blitz_beginner['Result'] == 'black', 'ECO'].value_counts().reset_index()
black_wins.columns = ['ECO', 'BlackWins']
df_opening_blitz_beginner = df_opening_blitz_beginner.merge(black_wins, on='ECO', how='left')

draws = df_blitz_beginner.loc[df_blitz_beginner['Result'] == 'draw', 'ECO'].value_counts().reset_index()
draws.columns = ['ECO', 'Draws']
df_opening_blitz_beginner = df_opening_blitz_beginner.merge(draws, on='ECO', how='left')

df_opening_blitz_beginner.fillna(0, inplace=True)

df_opening_blitz_beginner['WhiteWinPercentage'] = (df_opening_blitz_beginner['WhiteWins'] / df_opening_blitz_beginner['TotalGames']) * 100
df_opening_blitz_beginner['BlackWinPercentage'] = (df_opening_blitz_beginner['BlackWins'] / df_opening_blitz_beginner['TotalGames']) * 100
df_opening_blitz_beginner['DrawPercentage'] = (df_opening_blitz_beginner['Draws'] / df_opening_blitz_beginner['TotalGames']) * 100
df_opening_blitz_beginner.head(10)

,ECO,TotalGames,WhiteWins,BlackWins,Draws,WhiteWinPercentage,BlackWinPercentage,DrawPercentage
0,A00,1277,650.0,598.0,29.0,50.900548,46.828504,2.270948
1,B00,1056,488.0,538.0,30.0,46.212121,50.946970,2.840909
2,D00,782,413.0,351.0,18.0,52.813299,44.884910,2.301790
3,A40,638,324.0,304.0,10.0,50.783699,47.648903,1.567398
4,C41,626,315.0,284.0,27.0,50.319489,45.367412,4.313099
5,A04,474,352.0,111.0,11.0,74.261603,23.417722,2.320675
6,B01,454,212.0,224.0,18.0,46.696035,49.339207,3.964758
7,C44,432,190.0,202.0,40.0,43.981481,46.759259,9.259259
8,C20,409,207.0,192.0,10.0,50.611247,46.943765,2.444988
9,B20,370,163.0,188.0,19.0,44.054054,50.810811,5.135135


In [34]:
df_opening_blitz_intermediate = df_blitz_intermediate['ECO'].value_counts().reset_index()
df_opening_blitz_intermediate.columns = ['ECO', 'TotalGames']

white_wins = df_blitz_intermediate.loc[df_blitz_intermediate['Result'] == 'white', 'ECO'].value_counts().reset_index()
white_wins.columns = ['ECO', 'WhiteWins']
df_opening_blitz_intermediate = df_opening_blitz_intermediate.merge(white_wins, on='ECO', how='left')

black_wins = df_blitz_intermediate.loc[df_blitz_intermediate['Result'] == 'black', 'ECO'].value_counts().reset_index()
black_wins.columns = ['ECO', 'BlackWins']
df_opening_blitz_intermediate = df_opening_blitz_intermediate.merge(black_wins, on='ECO', how='left')

draws = df_blitz_intermediate.loc[df_blitz_intermediate['Result'] == 'draw', 'ECO'].value_counts().reset_index()
draws.columns = ['ECO', 'Draws']
df_opening_blitz_intermediate = df_opening_blitz_intermediate.merge(draws, on='ECO', how='left')

df_opening_blitz_intermediate.fillna(0, inplace=True)

df_opening_blitz_intermediate['WhiteWinPercentage'] = (df_opening_blitz_intermediate['WhiteWins'] / df_opening_blitz_intermediate['TotalGames']) * 100
df_opening_blitz_intermediate['BlackWinPercentage'] = (df_opening_blitz_intermediate['BlackWins'] / df_opening_blitz_intermediate['TotalGames']) * 100
df_opening_blitz_intermediate['DrawPercentage'] = (df_opening_blitz_intermediate['Draws'] / df_opening_blitz_intermediate['TotalGames']) * 100
df_opening_blitz_intermediate.head(10)

,ECO,TotalGames,WhiteWins,BlackWins,Draws,WhiteWinPercentage,BlackWinPercentage,DrawPercentage
0,A00,40257,19078.0,19818.0,1361.0,47.390516,49.228706,3.380778
1,A40,36050,18772.0,15926.0,1352.0,52.072122,44.177531,3.750347
2,D00,34821,18012.0,15292.0,1517.0,51.727406,43.916028,4.356566
3,B01,25682,11823.0,12736.0,1123.0,46.036134,49.591153,4.372712
4,B00,22860,11690.0,10367.0,803.0,51.137358,45.349956,3.512686
5,C00,22680,10109.0,11687.0,884.0,44.572310,51.529982,3.897707
6,B07,19743,9391.0,9540.0,812.0,47.566226,48.320924,4.112850
7,C41,18069,10038.0,7352.0,679.0,55.553711,40.688472,3.757817
8,D02,16493,9221.0,6595.0,677.0,55.908567,39.986661,4.104772
9,B20,16311,6802.0,8888.0,621.0,41.701919,54.490834,3.807247


In [35]:
df_opening_blitz_advanced = df_blitz_advanced['ECO'].value_counts().reset_index()
df_opening_blitz_advanced.columns = ['ECO', 'TotalGames']

white_wins = df_blitz_advanced.loc[df_blitz_advanced['Result'] == 'white', 'ECO'].value_counts().reset_index()
white_wins.columns = ['ECO', 'WhiteWins']
df_opening_blitz_advanced = df_opening_blitz_advanced.merge(white_wins, on='ECO', how='left')

black_wins = df_blitz_advanced.loc[df_blitz_advanced['Result'] == 'black', 'ECO'].value_counts().reset_index()
black_wins.columns = ['ECO', 'BlackWins']
df_opening_blitz_advanced = df_opening_blitz_advanced.merge(black_wins, on='ECO', how='left')

draws = df_blitz_advanced.loc[df_blitz_advanced['Result'] == 'draw', 'ECO'].value_counts().reset_index()
draws.columns = ['ECO', 'Draws']
df_opening_blitz_advanced = df_opening_blitz_advanced.merge(draws, on='ECO', how='left')

df_opening_blitz_advanced.fillna(0, inplace=True)

df_opening_blitz_advanced['WhiteWinPercentage'] = (df_opening_blitz_advanced['WhiteWins'] / df_opening_blitz_advanced['TotalGames']) * 100
df_opening_blitz_advanced['BlackWinPercentage'] = (df_opening_blitz_advanced['BlackWins'] / df_opening_blitz_advanced['TotalGames']) * 100
df_opening_blitz_advanced['DrawPercentage'] = (df_opening_blitz_advanced['Draws'] / df_opening_blitz_advanced['TotalGames']) * 100
df_opening_blitz_advanced.head(10)

,ECO,TotalGames,WhiteWins,BlackWins,Draws,WhiteWinPercentage,BlackWinPercentage,DrawPercentage
0,B01,673,272.0,272.0,129.0,40.416048,40.416048,19.167905
1,C50,562,322.0,179.0,61.0,57.295374,31.850534,10.854093
2,B30,472,120.0,288.0,64.0,25.423729,61.016949,13.559322
3,C30,353,192.0,97.0,64.0,54.390935,27.478754,18.130312
4,A00,337,142.0,123.0,72.0,42.136499,36.498516,21.364985
5,C48,153,45.0,86.0,22.0,29.411765,56.209150,14.379085
6,C00,142,58.0,61.0,23.0,40.845070,42.957746,16.197183
7,B11,142,78.0,37.0,27.0,54.929577,26.056338,19.014085
8,B40,140,64.0,52.0,24.0,45.714286,37.142857,17.142857
9,D02,125,38.0,60.0,27.0,30.400000,48.000000,21.600000


#### **Time Control: Rapid**

In [36]:
df_opening_rapid_beginner = df_rapid_beginner['ECO'].value_counts().reset_index()
df_opening_rapid_beginner.columns = ['ECO', 'TotalGames']

white_wins = df_rapid_beginner.loc[df_rapid_beginner['Result'] == 'white', 'ECO'].value_counts().reset_index()
white_wins.columns = ['ECO', 'WhiteWins']
df_opening_rapid_beginner = df_opening_rapid_beginner.merge(white_wins, on='ECO', how='left')

black_wins = df_rapid_beginner.loc[df_rapid_beginner['Result'] == 'black', 'ECO'].value_counts().reset_index()
black_wins.columns = ['ECO', 'BlackWins']
df_opening_rapid_beginner = df_opening_rapid_beginner.merge(black_wins, on='ECO', how='left')

draws = df_rapid_beginner.loc[df_rapid_beginner['Result'] == 'draw', 'ECO'].value_counts().reset_index()
draws.columns = ['ECO', 'Draws']
df_opening_rapid_beginner = df_opening_rapid_beginner.merge(draws, on='ECO', how='left')

df_opening_rapid_beginner.fillna(0, inplace=True)

df_opening_rapid_beginner['WhiteWinPercentage'] = (df_opening_rapid_beginner['WhiteWins'] / df_opening_rapid_beginner['TotalGames']) * 100
df_opening_rapid_beginner['BlackWinPercentage'] = (df_opening_rapid_beginner['BlackWins'] / df_opening_rapid_beginner['TotalGames']) * 100
df_opening_rapid_beginner['DrawPercentage'] = (df_opening_rapid_beginner['Draws'] / df_opening_rapid_beginner['TotalGames']) * 100
df_opening_rapid_beginner.head(10)

,ECO,TotalGames,WhiteWins,BlackWins,Draws,WhiteWinPercentage,BlackWinPercentage,DrawPercentage
0,D00,1441,775.0,618.0,48.0,53.782096,42.886884,3.331020
1,B00,915,594.0,277.0,44.0,64.918033,30.273224,4.808743
2,A40,867,457.0,384.0,26.0,52.710496,44.290657,2.998847
3,C41,859,448.0,388.0,23.0,52.153667,45.168801,2.677532
4,D02,853,421.0,398.0,34.0,49.355217,46.658851,3.985932
5,A00,765,323.0,425.0,17.0,42.222222,55.555556,2.222222
6,C00,666,338.0,306.0,22.0,50.750751,45.945946,3.303303
7,B10,497,235.0,243.0,19.0,47.283702,48.893360,3.822938
8,A45,463,229.0,216.0,18.0,49.460043,46.652268,3.887689
9,C20,451,228.0,207.0,16.0,50.554324,45.898004,3.547672


In [37]:
df_opening_rapid_intermediate = df_rapid_intermediate['ECO'].value_counts().reset_index()
df_opening_rapid_intermediate.columns = ['ECO', 'TotalGames']

white_wins = df_rapid_intermediate.loc[df_rapid_intermediate['Result'] == 'white', 'ECO'].value_counts().reset_index()
white_wins.columns = ['ECO', 'WhiteWins']
df_opening_rapid_intermediate = df_opening_rapid_intermediate.merge(white_wins, on='ECO', how='left')

black_wins = df_rapid_intermediate.loc[df_rapid_intermediate['Result'] == 'black', 'ECO'].value_counts().reset_index()
black_wins.columns = ['ECO', 'BlackWins']
df_opening_rapid_intermediate = df_opening_rapid_intermediate.merge(black_wins, on='ECO', how='left')

draws = df_rapid_intermediate.loc[df_rapid_intermediate['Result'] == 'draw', 'ECO'].value_counts().reset_index()
draws.columns = ['ECO', 'Draws']
df_opening_rapid_intermediate = df_opening_rapid_intermediate.merge(draws, on='ECO', how='left')

df_opening_rapid_intermediate.fillna(0, inplace=True)

df_opening_rapid_intermediate['WhiteWinPercentage'] = (df_opening_rapid_intermediate['WhiteWins'] / df_opening_rapid_intermediate['TotalGames']) * 100
df_opening_rapid_intermediate['BlackWinPercentage'] = (df_opening_rapid_intermediate['BlackWins'] / df_opening_rapid_intermediate['TotalGames']) * 100
df_opening_rapid_intermediate['DrawPercentage'] = (df_opening_rapid_intermediate['Draws'] / df_opening_rapid_intermediate['TotalGames']) * 100
df_opening_rapid_intermediate.head(10)

,ECO,TotalGames,WhiteWins,BlackWins,Draws,WhiteWinPercentage,BlackWinPercentage,DrawPercentage
0,A00,11245,4523.0,6159.0,563.0,40.222321,54.771009,5.006670
1,D00,5597,2842.0,2477.0,278.0,50.777202,44.255851,4.966947
2,C41,5546,3027.0,2311.0,208.0,54.579877,41.669672,3.750451
3,C00,5164,3083.0,1843.0,238.0,59.701782,35.689388,4.608830
4,B00,4193,2461.0,1596.0,136.0,58.693060,38.063439,3.243501
5,B01,3911,1888.0,1845.0,178.0,48.274099,47.174636,4.551266
6,B30,3815,1450.0,2224.0,141.0,38.007864,58.296199,3.695937
7,B20,3690,1550.0,2010.0,130.0,42.005420,54.471545,3.523035
8,C50,3661,1799.0,1731.0,131.0,49.139579,47.282163,3.578257
9,B07,3309,1710.0,1492.0,107.0,51.677244,45.089151,3.233605


In [38]:
df_opening_rapid_advanced = df_rapid_advanced['ECO'].value_counts().reset_index()
df_opening_rapid_advanced.columns = ['ECO', 'TotalGames']

white_wins = df_rapid_advanced.loc[df_rapid_advanced['Result'] == 'white', 'ECO'].value_counts().reset_index()
white_wins.columns = ['ECO', 'WhiteWins']
df_opening_rapid_advanced = df_opening_rapid_advanced.merge(white_wins, on='ECO', how='left')

black_wins = df_rapid_advanced.loc[df_rapid_advanced['Result'] == 'black', 'ECO'].value_counts().reset_index()
black_wins.columns = ['ECO', 'BlackWins']
df_opening_rapid_advanced = df_opening_rapid_advanced.merge(black_wins, on='ECO', how='left')

draws = df_rapid_advanced.loc[df_rapid_advanced['Result'] == 'draw', 'ECO'].value_counts().reset_index()
draws.columns = ['ECO', 'Draws']
df_opening_rapid_advanced = df_opening_rapid_advanced.merge(draws, on='ECO', how='left')

df_opening_rapid_advanced.fillna(0, inplace=True)

df_opening_rapid_advanced['WhiteWinPercentage'] = (df_opening_rapid_advanced['WhiteWins'] / df_opening_rapid_advanced['TotalGames']) * 100
df_opening_rapid_advanced['BlackWinPercentage'] = (df_opening_rapid_advanced['BlackWins'] / df_opening_rapid_advanced['TotalGames']) * 100
df_opening_rapid_advanced['DrawPercentage'] = (df_opening_rapid_advanced['Draws'] / df_opening_rapid_advanced['TotalGames']) * 100
df_opening_rapid_advanced.head(10)

,ECO,TotalGames,WhiteWins,BlackWins,Draws,WhiteWinPercentage,BlackWinPercentage,DrawPercentage
0,A00,132,60.0,56.0,16.0,45.454545,42.424242,12.121212
1,A45,107,55.0,42.0,10.0,51.401869,39.252336,9.345794
2,B21,101,44.0,45.0,12.0,43.564356,44.554455,11.881188
3,C00,90,44.0,36.0,10.0,48.888889,40.000000,11.111111
4,A40,82,28.0,44.0,10.0,34.146341,53.658537,12.195122
5,D45,67,27.0,38.0,2.0,40.298507,56.716418,2.985075
6,C41,65,28.0,27.0,10.0,43.076923,41.538462,15.384615
7,D31,62,25.0,25.0,12.0,40.322581,40.322581,19.354839
8,D03,50,20.0,25.0,5.0,40.000000,50.000000,10.000000
9,D02,49,18.0,19.0,12.0,36.734694,38.775510,24.489796


#### **Time Control: Bullet**

In [39]:
df_opening_bullet_beginner = df_bullet_beginner['ECO'].value_counts().reset_index()
df_opening_bullet_beginner.columns = ['ECO', 'TotalGames']

white_wins = df_bullet_beginner.loc[df_bullet_beginner['Result'] == 'white', 'ECO'].value_counts().reset_index()
white_wins.columns = ['ECO', 'WhiteWins']
df_opening_bullet_beginner = df_opening_bullet_beginner.merge(white_wins, on='ECO', how='left')

black_wins = df_bullet_beginner.loc[df_bullet_beginner['Result'] == 'black', 'ECO'].value_counts().reset_index()
black_wins.columns = ['ECO', 'BlackWins']
df_opening_bullet_beginner = df_opening_bullet_beginner.merge(black_wins, on='ECO', how='left')

draws = df_bullet_beginner.loc[df_bullet_beginner['Result'] == 'draw', 'ECO'].value_counts().reset_index()
draws.columns = ['ECO', 'Draws']
df_opening_bullet_beginner = df_opening_bullet_beginner.merge(draws, on='ECO', how='left')

df_opening_bullet_beginner.fillna(0, inplace=True)

df_opening_bullet_beginner['WhiteWinPercentage'] = (df_opening_bullet_beginner['WhiteWins'] / df_opening_bullet_beginner['TotalGames']) * 100
df_opening_bullet_beginner['BlackWinPercentage'] = (df_opening_bullet_beginner['BlackWins'] / df_opening_bullet_beginner['TotalGames']) * 100
df_opening_bullet_beginner['DrawPercentage'] = (df_opening_bullet_beginner['Draws'] / df_opening_bullet_beginner['TotalGames']) * 100
df_opening_bullet_beginner.head(10)

,ECO,TotalGames,WhiteWins,BlackWins,Draws,WhiteWinPercentage,BlackWinPercentage,DrawPercentage
0,A00,7,3.0,4.0,0.0,42.857143,57.142857,0.0
1,B00,3,1.0,2.0,0.0,33.333333,66.666667,0.0
2,B20,3,2.0,1.0,0.0,66.666667,33.333333,0.0
3,D00,3,1.0,2.0,0.0,33.333333,66.666667,0.0
4,C20,3,0.0,3.0,0.0,0.000000,100.000000,0.0
5,A40,3,2.0,1.0,0.0,66.666667,33.333333,0.0
6,B30,2,1.0,1.0,0.0,50.000000,50.000000,0.0
7,A34,2,0.0,2.0,0.0,0.000000,100.000000,0.0
8,A10,2,1.0,1.0,0.0,50.000000,50.000000,0.0
9,B50,1,1.0,0.0,0.0,100.000000,0.000000,0.0


In [40]:
df_opening_bullet_intermediate = df_bullet_intermediate['ECO'].value_counts().reset_index()
df_opening_bullet_intermediate.columns = ['ECO', 'TotalGames']

white_wins = df_bullet_intermediate.loc[df_bullet_intermediate['Result'] == 'white', 'ECO'].value_counts().reset_index()
white_wins.columns = ['ECO', 'WhiteWins']
df_opening_bullet_intermediate = df_opening_bullet_intermediate.merge(white_wins, on='ECO', how='left')

black_wins = df_bullet_intermediate.loc[df_bullet_intermediate['Result'] == 'black', 'ECO'].value_counts().reset_index()
black_wins.columns = ['ECO', 'BlackWins']
df_opening_bullet_intermediate = df_opening_bullet_intermediate.merge(black_wins, on='ECO', how='left')

draws = df_bullet_intermediate.loc[df_bullet_intermediate['Result'] == 'draw', 'ECO'].value_counts().reset_index()
draws.columns = ['ECO', 'Draws']
df_opening_bullet_intermediate = df_opening_bullet_intermediate.merge(draws, on='ECO', how='left')

df_opening_bullet_intermediate.fillna(0, inplace=True)

df_opening_bullet_intermediate['WhiteWinPercentage'] = (df_opening_bullet_intermediate['WhiteWins'] / df_opening_bullet_intermediate['TotalGames']) * 100
df_opening_bullet_intermediate['BlackWinPercentage'] = (df_opening_bullet_intermediate['BlackWins'] / df_opening_bullet_intermediate['TotalGames']) * 100
df_opening_bullet_intermediate['DrawPercentage'] = (df_opening_bullet_intermediate['Draws'] / df_opening_bullet_intermediate['TotalGames']) * 100
df_opening_bullet_intermediate.head(10)

,ECO,TotalGames,WhiteWins,BlackWins,Draws,WhiteWinPercentage,BlackWinPercentage,DrawPercentage
0,A00,3004,1578.0,1281.0,145.0,52.529960,42.643142,4.826897
1,A40,1084,552.0,490.0,42.0,50.922509,45.202952,3.874539
2,C00,1008,394.0,563.0,51.0,39.087302,55.853175,5.059524
3,B01,933,322.0,592.0,19.0,34.512326,63.451233,2.036442
4,A04,778,300.0,452.0,26.0,38.560411,58.097686,3.341902
5,B00,630,274.0,320.0,36.0,43.492063,50.793651,5.714286
6,D00,598,395.0,184.0,19.0,66.053512,30.769231,3.177258
7,A01,495,296.0,178.0,21.0,59.797980,35.959596,4.242424
8,B02,433,202.0,213.0,18.0,46.651270,49.191686,4.157044
9,B07,350,149.0,189.0,12.0,42.571429,54.000000,3.428571


In [41]:
df_opening_bullet_advanced = df_bullet_advanced['ECO'].value_counts().reset_index()
df_opening_bullet_advanced.columns = ['ECO', 'TotalGames']

white_wins = df_bullet_advanced.loc[df_bullet_advanced['Result'] == 'white', 'ECO'].value_counts().reset_index()
white_wins.columns = ['ECO', 'WhiteWins']
df_opening_bullet_advanced = df_opening_bullet_advanced.merge(white_wins, on='ECO', how='left')

black_wins = df_bullet_advanced.loc[df_bullet_advanced['Result'] == 'black', 'ECO'].value_counts().reset_index()
black_wins.columns = ['ECO', 'BlackWins']
df_opening_bullet_advanced = df_opening_bullet_advanced.merge(black_wins, on='ECO', how='left')

draws = df_bullet_advanced.loc[df_bullet_advanced['Result'] == 'draw', 'ECO'].value_counts().reset_index()
draws.columns = ['ECO', 'Draws']
df_opening_bullet_advanced = df_opening_bullet_advanced.merge(draws, on='ECO', how='left')

df_opening_bullet_advanced.fillna(0, inplace=True)

df_opening_bullet_advanced['WhiteWinPercentage'] = (df_opening_bullet_advanced['WhiteWins'] / df_opening_bullet_advanced['TotalGames']) * 100
df_opening_bullet_advanced['BlackWinPercentage'] = (df_opening_bullet_advanced['BlackWins'] / df_opening_bullet_advanced['TotalGames']) * 100
df_opening_bullet_advanced['DrawPercentage'] = (df_opening_bullet_advanced['Draws'] / df_opening_bullet_advanced['TotalGames']) * 100
df_opening_bullet_advanced.head(10)

,ECO,TotalGames,WhiteWins,BlackWins,Draws,WhiteWinPercentage,BlackWinPercentage,DrawPercentage
0,A00,193,59.0,98.0,36.0,30.569948,50.777202,18.652850
1,A40,37,23.0,7.0,7.0,62.162162,18.918919,18.918919
2,D02,36,15.0,10.0,11.0,41.666667,27.777778,30.555556
3,C67,36,2.0,5.0,29.0,5.555556,13.888889,80.555556
4,A04,36,27.0,6.0,3.0,75.000000,16.666667,8.333333
5,A46,31,16.0,13.0,2.0,51.612903,41.935484,6.451613
6,B12,26,11.0,4.0,11.0,42.307692,15.384615,42.307692
7,B00,26,20.0,4.0,2.0,76.923077,15.384615,7.692308
8,D00,24,10.0,10.0,4.0,41.666667,41.666667,16.666667
9,A45,23,6.0,5.0,12.0,26.086957,21.739130,52.173913


#### **Time Control: Classical**

In [42]:
df_opening_classical_beginner = df_classical_beginner['ECO'].value_counts().reset_index()
df_opening_classical_beginner.columns = ['ECO', 'TotalGames']

white_wins = df_classical_beginner.loc[df_classical_beginner['Result'] == 'white', 'ECO'].value_counts().reset_index()
white_wins.columns = ['ECO', 'WhiteWins']
df_opening_classical_beginner = df_opening_classical_beginner.merge(white_wins, on='ECO', how='left')

black_wins = df_classical_beginner.loc[df_classical_beginner['Result'] == 'black', 'ECO'].value_counts().reset_index()
black_wins.columns = ['ECO', 'BlackWins']
df_opening_classical_beginner = df_opening_classical_beginner.merge(black_wins, on='ECO', how='left')

draws = df_classical_beginner.loc[df_classical_beginner['Result'] == 'draw', 'ECO'].value_counts().reset_index()
draws.columns = ['ECO', 'Draws']
df_opening_classical_beginner = df_opening_classical_beginner.merge(draws, on='ECO', how='left')

df_opening_classical_beginner.fillna(0, inplace=True)

df_opening_classical_beginner['WhiteWinPercentage'] = (df_opening_classical_beginner['WhiteWins'] / df_opening_classical_beginner['TotalGames']) * 100
df_opening_classical_beginner['BlackWinPercentage'] = (df_opening_classical_beginner['BlackWins'] / df_opening_classical_beginner['TotalGames']) * 100
df_opening_classical_beginner['DrawPercentage'] = (df_opening_classical_beginner['Draws'] / df_opening_classical_beginner['TotalGames']) * 100
df_opening_classical_beginner.head(10)

,ECO,TotalGames,WhiteWins,BlackWins,Draws,WhiteWinPercentage,BlackWinPercentage,DrawPercentage
0,B00,15,11.0,3.0,1.0,73.333333,20.0,6.666667
1,D04,8,0.0,6.0,2.0,0.000000,75.0,25.000000
2,D00,5,4.0,0.0,1.0,80.000000,0.0,20.000000
3,D02,2,1.0,1.0,0.0,50.000000,50.0,0.000000
4,D05,2,0.0,2.0,0.0,0.000000,100.0,0.000000
5,C30,1,0.0,1.0,0.0,0.000000,100.0,0.000000
6,D25,1,1.0,0.0,0.0,100.000000,0.0,0.000000
7,C47,1,0.0,1.0,0.0,0.000000,100.0,0.000000
8,D06,1,1.0,0.0,0.0,100.000000,0.0,0.000000
9,C50,1,1.0,0.0,0.0,100.000000,0.0,0.000000


In [43]:
df_opening_classical_intermediate = df_classical_intermediate['ECO'].value_counts().reset_index()
df_opening_classical_intermediate.columns = ['ECO', 'TotalGames']

white_wins = df_classical_intermediate.loc[df_classical_intermediate['Result'] == 'white', 'ECO'].value_counts().reset_index()
white_wins.columns = ['ECO', 'WhiteWins']
df_opening_classical_intermediate = df_opening_classical_intermediate.merge(white_wins, on='ECO', how='left')

black_wins = df_classical_intermediate.loc[df_classical_intermediate['Result'] == 'black', 'ECO'].value_counts().reset_index()
black_wins.columns = ['ECO', 'BlackWins']
df_opening_classical_intermediate = df_opening_classical_intermediate.merge(black_wins, on='ECO', how='left')

draws = df_classical_intermediate.loc[df_classical_intermediate['Result'] == 'draw', 'ECO'].value_counts().reset_index()
draws.columns = ['ECO', 'Draws']
df_opening_classical_intermediate = df_opening_classical_intermediate.merge(draws, on='ECO', how='left')

df_opening_classical_intermediate.fillna(0, inplace=True)

df_opening_classical_intermediate['WhiteWinPercentage'] = (df_opening_classical_intermediate['WhiteWins'] / df_opening_classical_intermediate['TotalGames']) * 100
df_opening_classical_intermediate['BlackWinPercentage'] = (df_opening_classical_intermediate['BlackWins'] / df_opening_classical_intermediate['TotalGames']) * 100
df_opening_classical_intermediate['DrawPercentage'] = (df_opening_classical_intermediate['Draws'] / df_opening_classical_intermediate['TotalGames']) * 100
df_opening_classical_intermediate.head(10)

,ECO,TotalGames,WhiteWins,BlackWins,Draws,WhiteWinPercentage,BlackWinPercentage,DrawPercentage
0,A00,401,180.0,209.0,12.0,44.887781,52.119701,2.992519
1,B01,372,191.0,174.0,7.0,51.344086,46.774194,1.881720
2,B00,219,92.0,117.0,10.0,42.009132,53.424658,4.566210
3,B07,151,77.0,67.0,7.0,50.993377,44.370861,4.635762
4,C23,136,62.0,67.0,7.0,45.588235,49.264706,5.147059
5,C50,136,67.0,61.0,8.0,49.264706,44.852941,5.882353
6,C00,133,49.0,77.0,7.0,36.842105,57.894737,5.263158
7,C44,128,70.0,49.0,9.0,54.687500,38.281250,7.031250
8,D00,123,56.0,64.0,3.0,45.528455,52.032520,2.439024
9,A40,120,56.0,60.0,4.0,46.666667,50.000000,3.333333


In [44]:
df_opening_classical_advanced = df_classical_advanced['ECO'].value_counts().reset_index()
df_opening_classical_advanced.columns = ['ECO', 'TotalGames']

white_wins = df_classical_advanced.loc[df_classical_advanced['Result'] == 'white', 'ECO'].value_counts().reset_index()
white_wins.columns = ['ECO', 'WhiteWins']
df_opening_classical_advanced = df_opening_classical_advanced.merge(white_wins, on='ECO', how='left')

black_wins = df_classical_advanced.loc[df_classical_advanced['Result'] == 'black', 'ECO'].value_counts().reset_index()
black_wins.columns = ['ECO', 'BlackWins']
df_opening_classical_advanced = df_opening_classical_advanced.merge(black_wins, on='ECO', how='left')

draws = df_classical_advanced.loc[df_classical_advanced['Result'] == 'draw', 'ECO'].value_counts().reset_index()
draws.columns = ['ECO', 'Draws']
df_opening_classical_advanced = df_opening_classical_advanced.merge(draws, on='ECO', how='left')

df_opening_classical_advanced.fillna(0, inplace=True)

df_opening_classical_advanced['WhiteWinPercentage'] = (df_opening_classical_advanced['WhiteWins'] / df_opening_classical_advanced['TotalGames']) * 100
df_opening_classical_advanced['BlackWinPercentage'] = (df_opening_classical_advanced['BlackWins'] / df_opening_classical_advanced['TotalGames']) * 100
df_opening_classical_advanced['DrawPercentage'] = (df_opening_classical_advanced['Draws'] / df_opening_classical_advanced['TotalGames']) * 100
df_opening_classical_advanced.head(10)

,ECO,TotalGames,WhiteWins,BlackWins,Draws,WhiteWinPercentage,BlackWinPercentage,DrawPercentage
0,B22,30,12.0,8.0,10.0,40.000000,26.666667,33.333333
1,D02,25,13.0,3.0,9.0,52.000000,12.000000,36.000000
2,E33,20,12.0,4.0,4.0,60.000000,20.000000,20.000000
3,C47,19,10.0,4.0,5.0,52.631579,21.052632,26.315789
4,D35,19,3.0,5.0,11.0,15.789474,26.315789,57.894737
5,C42,19,4.0,3.0,12.0,21.052632,15.789474,63.157895
6,C65,18,7.0,4.0,7.0,38.888889,22.222222,38.888889
7,B45,17,6.0,3.0,8.0,35.294118,17.647059,47.058824
8,C50,17,7.0,6.0,4.0,41.176471,35.294118,23.529412
9,C55,16,2.0,4.0,10.0,12.500000,25.000000,62.500000


## **Summary**

## **References**
* https://lichess.org/faq#time-controls
* https://lichess.org/forum/general-chess-discussion/what-is-a-good-lichess-rating
* https://www.kaggle.com/datasets/datasnaek/chess